In [ ]:
#!pip install selenium

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')
service = Service("./chromedriver")  
driver = webdriver.Chrome(service=service, options=chrome_options)

In [3]:
URL = "https://naver.com"
driver.get(URL)
time.sleep(2)

In [17]:
# driver.find_element(By.CLASS_NAME, "MyView-module__link_login___HpHMW").click()
# time.sleep(2)

# ID = "아이디"
# PW = "비밀번호"
# driver.find_element(By.NAME, "id").send_keys(ID)
# driver.find_element(By.NAME, "pw").send_keys(PW)

# driver.find_element(By.NAME, "frmNIDLogin").submit()
# time.sleep(2)

In [4]:
URL = "https://cafe.naver.com/momsofsongdoifez"
driver.get(URL)

In [5]:
elem = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[6]/div[1]/div[2]/form/input")
elem.send_keys("아이모드")
driver.find_element(By.XPATH,"/html/body/div[4]/div/div[6]/div[1]/div[2]/form/button").click()
time.sleep(2)

In [6]:
# 수집할 카페 게시물의 링크주소를 저장할 리스트를 생성합니다.
post_list = []

URL = "https://cafe.naver.com/ArticleSearchList.nhn?search.clubid=26429203&search.media=0&search.searchdate=all&userDisplay=15&search.option=0&search.sortBy=date&search.searchBy=0&search.query=%BE%C6%C0%CC%B8%F0%B5%E5&search.viewtype=title&search.page="
page_limit = 4

for page_num in range(1, page_limit+1):

    driver.get(URL + str(page_num))
    driver.switch_to.frame(driver.find_element(By.NAME, "cafe_main"))
    elem = driver.find_elements(By.CLASS_NAME, "article")
    for e in elem:
        post_list.append(e.get_attribute("href"))

    driver.switch_to.default_content() 
    time.sleep(2)

In [7]:
print("수집된 게시물 URL 개수 :", len(post_list))

수집된 게시물 URL 개수 : 58


In [8]:
f = open("연구문제2-3", "w", encoding="utf-8")

for url in post_list[:]:
    try:
        driver.get(url)
        time.sleep(3)
        driver.switch_to.frame(driver.find_element(By.NAME, "cafe_main"))
        
        # 1. 게시물 제목
        title = driver.find_element(By.CLASS_NAME, "title_text").text.strip()
        # 2. 게시판 이름
        board_type = driver.find_element(By.CLASS_NAME, "link_board").text.strip()
        # 3. 게시물 작성일자
        datetime = driver.find_element(By.CLASS_NAME, "date").text
        # 4. 게시물 작성자(닉네임)
        user = driver.find_element(By.CLASS_NAME, "nickname").text
        # 5. 게시물 내용
        contents = driver.find_element(By.CLASS_NAME, "article_viewer").text.replace("\n", " ")

        f.write("POSTING" + "\t" + user + "\t" + datetime +
                "\t" + board_type + "\t" + title + "\t" + contents + "\n") 

        reply_list = driver.find_elements(By.CLASS_NAME, "comment_box")
        for reply in reply_list:
            # 6. 댓글 내용
            comment = reply.find_element(By.CLASS_NAME, "text_comment").text.replace("\n", " ")
            # 7. 댓글 작성자(닉네임)
            user_nik = reply.find_element(By.CLASS_NAME, "comment_nickname").text
            # 8. 댓글 작성일자
            reply_date = reply.find_element(By.CLASS_NAME, "comment_info_date").text

            f.write("COMMENT" + "\t" + user_nik + "\t" + reply_date + \
                    "\t" + board_type + "\t" + "None" + "\t" + comment + "\n")

        driver.switch_to.default_content()
        time.sleep(2)
    except:
        driver.switch_to.default_content()
        time.sleep(2)
        continue
f.close()

In [9]:
driver.close()